In [4]:
import json
from tqdm import tqdm
from malaysian_sft import detect_russian, detect_arabic, accept

In [10]:
files = [
    'chinese-qa.jsonl', 
    'indian-qa.jsonl'
]
instructions = []
for f in files:
    with open(f) as fopen:
        for l in tqdm(fopen):
            l = json.loads(l)

            if '.' in l:
                continue

            if detect_russian(l['question']):
                continue

            if detect_arabic(l['question']):
                continue

            if detect_russian(l['answer']):
                continue

            if detect_arabic(l['answer']):
                continue

            if 'answer' in l['answer'].lower():
                continue

            if 'question' in l['answer'].lower():
                continue

            if not accept(l['answer']):
                continue
                
            d = {
                'prompt_input': None,
                'input': l['question'],
                'output': l['answer'],
            }
            instructions.append(d)

10411it [00:04, 2250.36it/s]
10065it [00:08, 1146.80it/s]


In [11]:
len(instructions)

8258

In [12]:
import pandas as pd

df = pd.DataFrame(instructions)
df.to_parquet('prepared-llama3-70b-non-bumi-qa.parquet')

In [ ]:
from huggingface_hub import HfApi
api = HfApi()

api.upload_file(
    path_or_fileobj="prepared-llama3-70b-non-bumi-qa.parquet",
    path_in_repo='data/chatgpt4_malaysian_general_qa-00000-of-00001.parquet',
    repo_id="mesolitica/Malaysian-SFT",
    repo_type="dataset",
)